In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Let's see what is this data all about. 

#### As refer in Competition description, main task of this model is to determine the semantic similarity between phrases. 

### Some resources 

- [Semantic Textual similarity](https://www.sbert.net/docs/usage/semantic_textual_similarity.html)
- [Cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity#Angular_distance_and_similarity)
- [Sentence transformers](https://github.com/UKPLab/sentence-transformers)
- Sentence transformers models are also provided and listed on https://huggingface.co/sentence-transformers

In [ ]:
import os 
import numpy as np 
import pandas as pd
from typing import List, Dict, Union
from typing import Callable, TypeVar
from pathlib import Path

In [ ]:
HOME_PATH: str = '../input/us-patent-phrase-to-phrase-matching'
_files = list(Path(HOME_PATH).rglob("*.csv"))

In [ ]:
_train: pd.DataFrame = pd.read_csv(_files[1])

In [ ]:
_train.shape

#### Lets get more insights into data. 

### Lets look at some small chunks of data

In [ ]:
_train.shape

In [ ]:
_train.columns

In [ ]:
_train.dtypes

In [ ]:
_train.head(5)

### Unique id 
lets check for amount of unique values in different variables

In [ ]:
len(np.unique(_train.id))

In [ ]:
len(np.unique(_train.id)) == _train.shape[0]

So now we are sure that there are no duplicates and the number 

Anchor

In [ ]:
len(np.unique(_train.anchor))


There are 733 different categories of anchor, by anchor we should understand first phrase for which the similarity will be calculated. Let's check what is the distribution for anchors in train set.

In [ ]:
from collections import Counter

In [ ]:

_anchor_occurence = {k: v for k, v in sorted(Counter(_train.anchor).items(), key = lambda item: item[1], reverse = True)}

In [ ]:
_anchor_occurence

We can see that there are few classes for which occurences are greater than 100 and there are also anchor for which number of occurences is lower than 10. Let's look at distribution of that ocurence number. component composite coating is the most frequent anchor. 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize = (8, 8))
sns.displot(_anchor_occurence.values())

plt.grid()
plt.title("Number of occurences of different anchor classes")
plt.show()

# Target


In [ ]:
len(np.unique(_train.target))

In [ ]:
target_occurence = {k: v for k, v in sorted(Counter(_train.target).items(), key = lambda item: item[1], reverse = True)}

In [ ]:
target_occurence

The fun fact is that there are much more unique values for target and it's number of occurence in train dataset has much smaller amount of values for unique classes. 
We can see that there are strict differences between its distribution

In [ ]:

plt.figure(figsize = (10, 10))
sns.displot(target_occurence.values(), bins = 10)

plt.grid()
plt.title("Number of occurence for target variable")
plt.show()


### Target distribution

Let's show how much length of target variable is correlated with its score values.
There are 4 possible levels of target variblaes, which can jump by .25 and have a range from 0. to 1.

### Score distribution

In [ ]:
_score_count =  Counter(_train.score)

In [ ]:
_score_count

In [ ]:
{k: v/_train.shape[0] for k, v in _score_count.items()}

We can observe value .5 for score variable is most common and consist of .33 of all ocurrences in dataset. 

### Let's look at length of anchors and lengths of target vs score that it gets.
Just show if its correlated or not.

In [ ]:
_train['ratio'] = _train.target / _train


In [ ]:

_train['target_len']  = _train.target.str.len()
_train['anchor_len']  = _train.anchor.str.len()

In [ ]:
sns.displot(_train.target_len)

### Lets compare it with an anchor length

In [ ]:
sns.displot(_train.anchor_len)

In [ ]:
# Lets create a ratio anchor_len/target_len and check for its linear correlation with target variable.

In [ ]:
_train['tar_anch_ratio'] = _train.anchor_len/_train.target_len

In [ ]:
np.corrcoef(_train.tar_anch_ratio, _train.score)

There is wrong assumption with using linear correlation coeff  because our target variable has only like 5 levels.